### Import Libraries

In [40]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

2025-06-14 17:22:08.892301: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-14 17:22:10.662332: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749936131.319880    2998 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749936131.603600    2998 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-14 17:22:13.601326: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

## Preprocessing

### Read data for model

In [21]:
train_df = pd.read_csv("CBIS-DDSM_Clean_Data/train_descriptions.csv")
test_df = pd.read_csv("CBIS-DDSM_Clean_Data/test_descriptions.csv")

In [22]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2864 entries, 0 to 2863
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   breast_density    2864 non-null   int64 
 1   breast_side       2864 non-null   object
 2   image_view        2864 non-null   object
 3   abnormality_type  2864 non-null   object
 4   pathology         2864 non-null   object
 5   series_uid        2864 non-null   object
 6   images_new_paths  2864 non-null   object
 7   image_type        2864 non-null   object
dtypes: int64(1), object(7)
memory usage: 179.1+ KB


In [23]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 704 entries, 0 to 703
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   breast_density    704 non-null    int64 
 1   breast_side       704 non-null    object
 2   image_view        704 non-null    object
 3   abnormality_type  704 non-null    object
 4   pathology         704 non-null    object
 5   series_uid        704 non-null    object
 6   images_new_paths  704 non-null    object
 7   image_type        704 non-null    object
dtypes: int64(1), object(7)
memory usage: 44.1+ KB


In [24]:
train_df.iloc[0]

breast_density                                                      3
breast_side                                                      LEFT
image_view                                                         CC
abnormality_type                                                 mass
pathology                                                   MALIGNANT
series_uid          1.3.6.1.4.1.9590.100.1.2.342386194811267636608...
images_new_paths    1.3.6.1.4.1.9590.100.1.2.342386194811267636608...
image_type                                      full mammogram images
Name: 0, dtype: object

In [25]:
print("Size of training data", len(train_df))
print("Size of testing data", len(test_df))

Size of training data 2864
Size of testing data 704


### Encode labels

In [34]:
# create dataframe and transform(encodes) pathology labels
train_full_df = train_df
test_full_df = test_df
print("Non Encoded: ")
print(train_full_df["pathology"].unique())
print(test_full_df["pathology"].unique())
print()

train_full_df["label"] = LabelEncoder().fit_transform(train_full_df["pathology"])
test_full_df["label"] = LabelEncoder().fit_transform(test_full_df["pathology"])

print("Encoded: ")
print(train_full_df["label"].unique())
print(test_full_df["label"].unique())

Non Encoded: 
['MALIGNANT' 'BENIGN' 'BENIGN_WITHOUT_CALLBACK']
['MALIGNANT' 'BENIGN' 'BENIGN_WITHOUT_CALLBACK']

Encoded: 
[2 0 1]
[2 0 1]


### Split Training data into validation and training sets

In [37]:
train_data, val_data = train_test_split(train_full_df, 
                                        test_size=0.15, 
                                        stratify=train_full_df["label"], 
                                        random_state=42
                                       )
test_data = test_full_df.copy()

print("Train set", len(train_data))
print("Validation set", len(val_data))
print("Test set", len(test_data))

Train set 2434
Validation set 430
Test set 704


## Model Architecture 